In [1]:
#This is to turn any file or query into a pandas dataframe
import pandas as pd
import os
import sqlite3
import glob
from zipfile import ZipFile

def file_or_sql_to_dataframe(source, save_csv=False, csv_filename=None, sql_connection=None):
    """
    Reads a file (txt, json, excel, or sql query), all files in a directory, or a zip archive and converts it into a pandas DataFrame.
    Optionally saves the DataFrame as a CSV file.

    Args:
        source (str): Path to the file, directory, zip archive, or SQL query.
        save_csv (bool): Whether to save the DataFrame as a CSV file.
        csv_filename (str): Filename to save the CSV file as (optional).
        sql_connection (str or connection object): SQL connection string or connection object.

    Returns:
        pd.DataFrame: DataFrame created from the file(s) or SQL query.
    """
    def read_files_from_dir(directory):
        """Read and concatenate all txt and json files in the directory into a DataFrame"""
        file_paths = glob.glob(os.path.join(directory, '*.txt')) + glob.glob(os.path.join(directory, '*.json'))
        dfs = []
        for file_path in file_paths:
            file_extension = os.path.splitext(file_path)[-1].lower()
            if file_extension == '.txt':
                df = pd.read_csv(file_path, delimiter='\t', encoding='utf-8')
            elif file_extension == '.json':
                df = pd.read_json(file_path)
            else:
                continue
            dfs.append(df)
        return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

    if sql_connection is not None:
        conn = sqlite3.connect(sql_connection) if isinstance(sql_connection, str) else sql_connection
        df = pd.read_sql(source, conn)
    elif os.path.isdir(source):
        df = read_files_from_dir(source)
    elif os.path.splitext(source)[-1].lower() == '.zip':
        with ZipFile(source, 'r') as zip_ref:
            temp_dir = 'temp_extracted_files'
            os.makedirs(temp_dir, exist_ok=True)
            zip_ref.extractall(temp_dir)
            df = read_files_from_dir(temp_dir)
            for file in os.listdir(temp_dir):
                os.remove(os.path.join(temp_dir, file))
            os.rmdir(temp_dir)
    else:
        if source.endswith('.txt'):
            df = pd.read_csv(source, delimiter='\t', encoding='utf-8')
        elif source.endswith('.json'):
            df = pd.read_json(source)
        elif source.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(source)
        elif source.endswith('.csv'):
            df = pd.read_csv(source)
        else:
            raise ValueError("Unsupported file type. File must be .txt, .json, .xls, .xlsx, .zip, or SQL query.")

    if save_csv and not df.empty:
        csv_filename = csv_filename or os.path.splitext(source)[0] + '.csv'
        df.to_csv(csv_filename, index=False)
        print(f"Data saved as CSV file: {csv_filename}")

    return df

# Example usage
# df = file_or_sql_to_dataframe('path/to/your/file.txt', save_csv=True)
# df = file_or_sql_to_dataframe('path/to/your/directory')
# df = file_or_sql_to_dataframe('path/to/your/zipfile.zip')


In [2]:
data_test = file_or_sql_to_dataframe("C:/Users/dkim//Downloads/dataverse_files.zip")
data_test.shape

(50438, 40)

In [3]:
data_test = file_or_sql_to_dataframe("C:/Users/dkim//Downloads/Emergency_Connectivity_Fund_FCC_Form_471_20231229 - Copy.txt")
data_test.shape

(120552, 82)

In [4]:
#
ecf_data = pd.read_csv("C:/Users/dkim//Downloads/Emergency_Connectivity_Fund_FCC_Form_471_20231229.csv")
ecf_data.describe()
ecf_data = ecf_data[ecf_data['Form Version'] == 'Current']

ecf_data_sample = ecf_data.sample(frac=0.5) #this is to select on 50% of the data
ecf_data_sample.shape

(31463, 82)

In [5]:
#developing a unique identifier
ecf_data_sample['unique_key'] = ecf_data_sample['Funding Request Number (FRN)'].astype(str) + '_' + ecf_data_sample['FRN Line Item ID'].astype(str) + '_' + ecf_data_sample['One-time Unit Cost'].astype(str) + '_' + ecf_data_sample['One-time Unit Quantity'].astype(str)


In [6]:
ecf_data_sample['unique_key']

44041     ECF2190007284_1_327.0_250
39517         ECF2290010096_2_0.0_0
63522         ECF2190007775_1_0.0_0
114165        ECF2190027746_2_0.0_1
18920     ECF2290007014_1_394.25_20
                    ...            
102379        ECF2190033360_1_0.0_0
39564      ECF2190004899_3_400.0_20
114121    ECF2190027662_3_335.19_25
4785        ECF2290010869_3_264.0_6
84860      ECF2190013111_3_50.0_110
Name: unique_key, Length: 31463, dtype: object

In [7]:
ecf_data_sample_selected = ecf_data_sample.loc[:, ['Application Number', 'unique_key', 'Total Funding Commitment Request Amount', 'Product Type', 'Urban/ Rural Status', 'Monthly Recurring Unit Cost', 'Total Student Count']]
ecf_data_sample_selected.shape 

(31463, 7)

In [10]:
metadata = SingleTableMetadata() #this is set up before hand to prepare for the evalutaion testing
metadata.detect_from_dataframe(data=ecf_data_sample_selected)

In [9]:
from sdv.single_table import TVAESynthesizer, GaussianCopulaSynthesizer, CTGANSynthesizer, CopulaGANSynthesizer
from sdv.lite import SingleTablePreset
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import get_column_plot, evaluate_quality, get_column_pair_plot

def generate_synthetic_data(dataset, sdv_type, unique_id, num_samples):
    """
    Generates synthetic data using the specified SDV synthesizer.

    Args:
        dataset (pd.DataFrame): The dataset to base the synthetic data on.
        sdv_type (str): Type of SDV synthesizer to use ('TVAE', 'GaussianCopula', 'CopulaGAN', 'fast_ml', or 'CTGAN').
        unique_id (str): Column name to set as a primary key in the dataset.
        num_samples (int): Number of synthetic data rows to generate.

    Returns:
        pd.DataFrame: The generated synthetic dataset.
    """
    # Create metadata object and detect from dataframe
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=dataset)
    
    # Update metadata for the unique key column
    metadata.update_column(column_name=unique_id, sdtype='id')
    metadata.set_primary_key(column_name=unique_id)
    
    # Validate metadata
    metadata.validate_data(data=dataset) #if there is an error it will provide an in depth look in the error
    
    # Select the synthesizer based on sdv_type
    if sdv_type == 'TVAE':
        synthesizer = TVAESynthesizer(
                        metadata, # required
                        enforce_min_max_values=True,
                        enforce_rounding=True,
                        epochs=500) 
    elif sdv_type == 'GaussianCopula':
        synthesizer = GaussianCopulaSynthesizer(
                        metadata, 
                        enforce_min_max_values=True,
                        enforce_rounding=True,
                        #numerical_distributions={
                        #    'amenities_fee': 'beta',
                        #    'checkin_date': 'uniform'
                        #        },
                        #default_distribution='norm' #lets you determine what distrubution your working  with, read file for more information: https://docs.sdv.dev/sdv/single-table-data/modeling/synthesizers/gaussiancopulasynthesizer
                        )
    elif sdv_type == 'CTGAN':
        synthesizer = CTGANSynthesizer(
                        metadata, 
                        enforce_rounding=True,
                        epochs=500,
                        verbose=True)
    elif sdv_type == 'CopulaGAN':
        synthesizer = CopulaGANSynthesizer(
                        metadata, 
                        enforce_min_max_values=True,
                        enforce_rounding=True,
                     #   numerical_distributions={
                     #       'amenities_fee': 'beta',
                     #       'checkin_date': 'uniform'
                     #                       },
                        epochs=500,
                        verbose=True)
    elif sdv_type == 'fast_ml':
        synthesizer = SingleTablePreset(metadata, name='FAST_ML')
    else:
        return "Error: Invalid SDV synthesizer type. Choose 'TVAE', 'GaussianCopula', 'CopulaGAN', 'fast_ml', or 'CTGAN'."

    # Fit the synthesizer on the dataset
    synthesizer.fit(dataset)
    
    # Generate synthetic data
    synthetic_data = synthesizer.sample(num_rows=num_samples)
    
    return synthetic_data

# Example usage
# synthetic_dataset = generate_synthetic_data(ecf_data_sample_selected, 'TVAE', 'unique_key', 25000)


In [11]:
synthetic_dataset = generate_synthetic_data(ecf_data_sample_selected, 'TVAE',"unique_key", 25000)

In [ ]:
synthetic_dataset = generate_synthetic_data(ecf_data_sample_selected, 'GaussianCopula',"unique_key", 25000)

In [12]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic_report = run_diagnostic( #this is to ensure that the model has worked properly. Anything lower than a 100 is a problem and should be taken care of
    real_data=ecf_data_sample_selected,
    synthetic_data=synthetic_dataset,
    metadata=metadata)

Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<00:00, 499.14it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%


In [13]:
quality_report = evaluate_quality( 
    real_data=ecf_data_sample_selected, #also this is the model without hyperparameter trained
    synthetic_data=synthetic_dataset,
    metadata=metadata #63% overal score, column shapes is 87%, column pair trends is 38% if the model is not trained hyperparameter
)

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 21/21 [00:00<00:00, 56.76it/s] 

Overall Score: 63.6%

Properties:
- Column Shapes: 88.66%
- Column Pair Trends: 38.53%


In [14]:
quality_report.get_details(property_name='Column Shapes') #shows the accuracy in details

,Column,Metric,Score
0,Total Funding Commitment Request Amount,KSComplement,0.932573
1,Product Type,TVComplement,0.919209
2,Urban/ Rural Status,TVComplement,0.983609
3,Monthly Recurring Unit Cost,KSComplement,0.669842
4,Total Student Count,KSComplement,0.927928


In [ ]:
from sdv.evaluation.single_table import get_column_plot
import plotly.io as pio

# This ensures Plotly plots show up in Jupyter notebooks
pio.renderers.default = 'notebook'

fig = get_column_plot(
    real_data=ecf_data_sample_selected,
    synthetic_data=synthetic_dataset,
    metadata=metadata,
    column_name='Total Funding Commitment Request Amount'
)

# Show the figure using Plotly's show method
fig.show()



In [ ]:
from sdv.evaluation.single_table import get_column_pair_plot

fig = get_column_pair_plot(
    real_data=ecf_data_sample_selected,
    synthetic_data=synthetic_dataset,
    metadata=metadata,
    column_names=['Total Funding Commitment Request Amount', 'Total Student Count'],
    )
    
fig.show()